In [2]:
# !pip install transformers torch

In [3]:
from transformers import BertTokenizer, BertForMaskedLM
import torch 

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
text = '''Independence Day is celebrated annually on 15 August as a national holiday in India commemorating the nation's independence from the United Kingdom on 15 August 1947, the day when the provisions of the 1947 Indian Independence Act, which transferred legislative sovereignty to the Indian Constituent Assembly, came into effect.India retained King George VI as head of state until its transition to a republic, when the nation adopted the Constitution of India on 26 January 1950 (celebrated as Indian Republic Day) and replaced the dominion prefix, Dominion of India, with the enactment of the sovereign law Constitution of India.India attained independence following the Independence Movement noted for largely non-violent resistance and civil disobedience.'''

In [23]:
text = text.split('.')

In [24]:
text = text[0:len(text)-1]
text = tuple(text)

In [26]:
inputs = tokenizer(text, return_tensors='pt', padding = True)

In [28]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [29]:
inputs.input_ids

tensor([[  101,  4336,  2154,  2003,  6334,  6604,  2006,  2321,  2257,  2004,
          1037,  2120,  6209,  1999,  2634, 20646,  1996,  3842,  1005,  1055,
          4336,  2013,  1996,  2142,  2983,  2006,  2321,  2257,  4006,  1010,
          1996,  2154,  2043,  1996,  8910,  1997,  1996,  4006,  2796,  4336,
          2552,  1010,  2029,  4015,  4884, 12601,  2000,  1996,  2796, 13794,
          3320,  1010,  2234,  2046,  3466,   102,     0],
        [  101,  2634,  6025,  2332,  2577,  6819,  2004,  2132,  1997,  2110,
          2127,  2049,  6653,  2000,  1037,  3072,  1010,  2043,  1996,  3842,
          4233,  1996,  4552,  1997,  2634,  2006,  2656,  2254,  3925,  1006,
          6334,  2004,  2796,  3072,  2154,  1007,  1998,  2999,  1996, 13738,
         17576,  1010, 13738,  1997,  2634,  1010,  2007,  1996, 26465,  1997,
          1996, 11074,  2375,  4552,  1997,  2634,   102],
        [  101,  2634, 12754,  4336,  2206,  1996,  4336,  2929,  3264,  2005,
          432

In [32]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [33]:
rand = torch.rand(inputs.input_ids.shape)

In [35]:
rand.shape

torch.Size([3, 57])

In [39]:
mask_arr = (rand<0.15) * (inputs.input_ids!=101) * (inputs.input_ids!=102) * (inputs.input_ids!=0)
mask_arr

tensor([[False, False, False,  True, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False,  True, False,  True, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False,  True, False, False, False, False,
         False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False,  True, False,
         False, False, False, False, False, False, False, False, False,  True,
         False, False, False, False, False, False, False, False, False, False,
         False,  True, False, False, False, False, False, False, False, False,
          True,  True, False,  True, False, False, False, False, False, False,
          True, False, False, False, False,  True, False],
        [False, False, False, False, False, False, False, False, False, False,
         Fals

In [43]:
selection = torch.flatten(mask_arr[0].nonzero()).tolist()

In [51]:
inputs.input_ids[0, selection] = 103

In [52]:
inputs.input_ids

tensor([[  101,  4336,  2154,   103,  6334,   103,  2006,  2321,  2257,  2004,
          1037,  2120,  6209,  1999,  2634, 20646,  1996,  3842,  1005,  1055,
          4336,  2013,  1996,   103,  2983,   103,  2321,  2257,  4006,  1010,
          1996,  2154,  2043,  1996,  8910,  1997,  1996,  4006,  2796,  4336,
          2552,  1010,  2029,  4015,  4884,   103,  2000,  1996,  2796, 13794,
          3320,  1010,  2234,  2046,  3466,   102,     0],
        [  101,  2634,  6025,  2332,  2577,  6819,  2004,  2132,  1997,  2110,
          2127,  2049,  6653,  2000,  1037,  3072,  1010,  2043,  1996,  3842,
          4233,  1996,  4552,  1997,  2634,  2006,  2656,  2254,  3925,  1006,
          6334,  2004,  2796,  3072,  2154,  1007,  1998,  2999,  1996, 13738,
         17576,  1010, 13738,  1997,  2634,  1010,  2007,  1996, 26465,  1997,
          1996, 11074,  2375,  4552,  1997,  2634,   102],
        [  101,  2634, 12754,  4336,  2206,  1996,  4336,  2929,  3264,  2005,
          432

In [65]:
output = model(**inputs)

In [66]:
output

MaskedLMOutput(loss=tensor(5.3161, grad_fn=<NllLossBackward>), logits=tensor([[[ -6.7807,  -6.7796,  -6.8050,  ...,  -6.1793,  -5.9404,  -4.3070],
         [ -9.7692,  -9.6230,  -9.5597,  ..., -10.5110,  -8.2500,  -8.6564],
         [-13.5322, -13.8062, -13.5406,  ..., -11.1229, -11.8846, -13.1268],
         ...,
         [ -7.5839,  -7.3558,  -7.3843,  ...,  -6.2852,  -7.0611,  -6.7315],
         [-13.8872, -13.6713, -13.6778,  ..., -12.0400, -12.3265, -11.2298],
         [ -8.4752,  -8.6548,  -8.5007,  ...,  -8.0214,  -8.7565,  -4.4111]],

        [[ -7.9836,  -8.0838,  -7.9971,  ...,  -7.2617,  -7.0774,  -4.8206],
         [ -8.5081,  -8.5799,  -8.5730,  ...,  -8.5265,  -7.4636,  -6.2909],
         [ -5.3238,  -5.1884,  -4.8106,  ...,  -5.8731,  -2.1354,  -3.0779],
         ...,
         [-13.3884, -14.1057, -13.3856,  ..., -13.0293,  -9.7378,  -9.6451],
         [-11.1971, -11.7242, -11.7200,  ..., -11.2895,  -9.1043,  -8.0491],
         [-11.5078, -11.5522, -11.5823,  ..., -10.567